In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plot 

In [ ]:
df = pd.read_csv('./datasets/Electric_Vehicle_Charging_Station_Data.csv')
df.head()

In [ ]:
# Selecting the columns to keep 
columns_to_keep = ['ObjectId2','Address' , 'Start_Date___Time', 'End_Date___Time', 'Total_Duration__hh_mm_ss_' , 'Charging_Time__hh_mm_ss_' , 'Energy__kWh_']

# Filtering the dataset
df_filtered = df[columns_to_keep].copy()  # Make a copy to avoid the SettingWithCopyWarning

In [ ]:
# On checking it is clear that all the elements in the df_filtered are 'str' and not datetime objects
# Converting them to datetime and timedelta

df_filtered.loc[:, 'Start_Date___Time'] = pd.to_datetime(df_filtered['Start_Date___Time'])
df_filtered.loc[:, 'End_Date___Time'] = pd.to_datetime(df_filtered['End_Date___Time'])
df_filtered.loc[:, 'Total_Duration__hh_mm_ss_'] = pd.to_timedelta(df_filtered['Total_Duration__hh_mm_ss_'])
df_filtered.loc[:, 'Charging_Time__hh_mm_ss_'] = pd.to_timedelta(df_filtered['Charging_Time__hh_mm_ss_'])

# Filter out rows where charging time is zero
df_filtered = df_filtered[df_filtered['Charging_Time__hh_mm_ss_'] > pd.Timedelta('0 seconds')]

df_filtered.head()

In [ ]:
# Checking that in all cases charging time < total duration 

rows_with_greater_charging_time = []
for index, row in df_filtered.iterrows():
    if row['Charging_Time__hh_mm_ss_'] > row['Total_Duration__hh_mm_ss_']:
        rows_with_greater_charging_time.append(index)

print(rows_with_greater_charging_time)

### Currently just conducting analysis of all the EV charging stations together

In [ ]:
# Create a new DataFrame with charging data per minute
charging_per_minute_df = pd.DataFrame()

# Set the frequency to 'T' for minutes directly
start_time = min(df_filtered['Start_Date___Time'])
end_time = max(df_filtered['End_Date___Time'])

# Create a 'Timestamp' column with the pd.date_range
charging_per_minute_df['Timestamp'] = pd.date_range(start=start_time, end=end_time, freq='T')

# Initialize charging per minute column with zeros
charging_per_minute_df['Charging_Per_Minute'] = 0.0

# Iterate through each row in the filtered DataFrame
for index, row in df_filtered.iterrows():
    start_time = row['Start_Date___Time']
    end_time = start_time + row['Charging_Time__hh_mm_ss_']
    charging_time_seconds = row['Charging_Time__hh_mm_ss_'].total_seconds()

    # Update the mask to consider start_time to end_time range
    mask = (charging_per_minute_df['Timestamp'] >= start_time) & (charging_per_minute_df['Timestamp'] <= end_time)

    # Calculate charging rate per second
    charging_rate_per_second = row['Energy__kWh_'] / max(charging_time_seconds, 1e-6)

    # Loop to update the charging rate for each minute
    for idx in charging_per_minute_df.loc[mask].index[:-2]:
        charging_per_minute_df.loc[idx, 'Charging_Per_Minute'] += charging_rate_per_second * 60
        
    print(row)

    # Update the charging rate for the second last minute if there are at least two indices
    if len(charging_per_minute_df.loc[mask].index) >= 2:
        second_last_timestamp = charging_per_minute_df.loc[mask].index[-2]
        last_minute_seconds = charging_time_seconds % 60
        charging_per_minute_df.loc[second_last_timestamp, 'Charging_Per_Minute'] += (min(60, last_minute_seconds)) * charging_rate_per_second
    else:
        # If there is only one timestamp, set its charging rate
        charging_per_minute_df.loc[mask, 'Charging_Per_Minute'] += charging_rate_per_second * charging_time_seconds


# Convert charging per second data to charging per day
charging_per_day = charging_per_second_df.resample('D', on='Timestamp').sum()

### The purpose of this notebook is just to showcase how we are aggregating the data

In [ ]:
charging_per_minute_df

In [ ]:
# Now, you can use resample
charging_per_15_minutes_df = charging_per_minute_df.resample('15T').sum()

# Reset the index if needed
charging_per_15_minutes_df.reset_index(inplace=True)

# Rename the column
charging_per_15_minutes_df = charging_per_15_minutes_df.rename(columns={'Charging_Per_Minute': 'Charging_Per_15_Minutes'})

charging_per_15_minutes_df

In [8]:
charging_per_15_minutes_df = pd.read_csv("datasets/colorado_15_minutes.csv")
charging_per_15_minutes_df['Timestamp'] = pd.to_datetime(charging_per_15_minutes_df['Timestamp'], format='%Y-%m-%d %H:%M:%S')

# Convert charging per 15 mins data to charging per day
charging_per_day = charging_per_15_minutes_df.resample('D', on='Timestamp').sum()
charging_per_day.rename(columns={'Charging_Per_15_Minutes': 'Charging_per_day'}, inplace=True)
charging_per_day_reset = charging_per_day.reset_index()
charging_per_day

,Charging_per_day
Timestamp,
2018-01-01,12.902014
2018-01-02,27.783363
2018-01-03,28.840734
2018-01-04,96.127050
2018-01-05,8.299380
...,...
2023-12-04,0.000000
2023-12-05,0.000000
2023-12-06,405.218349


In [3]:
# Loading the weather data

weather_df = pd.read_csv('datasets/Weather_data_boulder.csv')
weather_df.head()

,YEAR,MONTH,DAY,MAX. TEMP(F),MIN. TEMP(F),Precipitation (Inches),Snowfall (Inches),Snow Cover (Inches)
0,2018,1,1,30,12,0.0,0.0,-999
1,2018,1,2,46,12,0.0,0.0,-999
2,2018,1,3,50,20,0.0,0.0,-999
3,2018,1,4,52,24,0.0,0.0,-999
4,2018,1,5,62,25,0.0,0.0,-999


In [4]:
# Remove the 29th February data for 2018 , 2019 , 2021 , 2022 , 2023

for index, row in weather_df.iterrows():
    year = row['YEAR']
    month = row['MONTH']
    day = row['DAY']

    # Check if it's 29th February in a non-leap year
    if month == 2 and day == 29 and not pd.to_datetime(year, format='%Y').is_leap_year:
        weather_df = weather_df.drop(index)

# Resetting the index after removing rows
weather_df = weather_df.reset_index(drop=True)

In [10]:
# Merging both the datasets

# Convert the date columns in both dataframes to datetime format
weather_df['DATE'] = pd.to_datetime(weather_df[['YEAR', 'MONTH', 'DAY']])

# Merge the two datasets based on the complete timestamp
merged_df = pd.merge(charging_per_day_reset, weather_df, how='left', left_on='Timestamp', right_on='DATE')

# Drop the duplicate date columns from weather_df
merged_df = merged_df.drop(columns=['DATE' , 'YEAR', 'MONTH', 'DAY' ])

# Extract the day of the week as an integer (Monday=1, Sunday=7)
merged_df['DayOfWeek'] = merged_df['Timestamp'].dt.dayofweek+1  

# One-hot encode the day of the week
merged_df = pd.get_dummies(merged_df, columns=['DayOfWeek'])

day_of_week_mapping = {
    1: 'Monday',
    2: 'Tuesday',
    3: 'Wednesday',
    4: 'Thursday',
    5: 'Friday',
    6: 'Saturday',
    7: 'Sunday'
}

merged_df.rename(columns={f'DayOfWeek_{i}': day_of_week_mapping[i] for i in range(1, 8)}, inplace=True)

# Replace -999 with 0 in 'Snowfall' and 'Precipitation' columns since trace values of rainfall and snowfall 
# should not have an impact on EV load
merged_df['Snowfall (Inches)'] = merged_df['Snowfall (Inches)'].replace(-999, 0)
merged_df['Snow Cover (Inches)'] = merged_df['Snow Cover (Inches)'].replace(-999, 0)

merged_df['Precipitation (Inches)'] = merged_df['Precipitation (Inches)'].replace(-999, 0)

merged_df

,Timestamp,Charging_per_day,MAX. TEMP(F),MIN. TEMP(F),Precipitation (Inches),Snowfall (Inches),Snow Cover (Inches),Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,2018-01-01,12.902014,30,12,0.0,0.0,0,True,False,False,False,False,False,False
1,2018-01-02,27.783363,46,12,0.0,0.0,0,False,True,False,False,False,False,False
2,2018-01-03,28.840734,50,20,0.0,0.0,0,False,False,True,False,False,False,False
3,2018-01-04,96.127050,52,24,0.0,0.0,0,False,False,False,True,False,False,False
4,2018-01-05,8.299380,62,25,0.0,0.0,0,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2163,2023-12-04,0.000000,59,24,0.0,0.0,0,True,False,False,False,False,False,False
2164,2023-12-05,0.000000,65,26,0.0,0.0,0,False,True,False,False,False,False,False
2165,2023-12-06,405.218349,65,34,0.0,0.0,0,False,False,True,False,False,False,False
2166,2023-12-07,808.579063,59,37,0.0,0.0,0,False,False,False,True,False,False,False


In [11]:
# Adding seasonality indicators , weekend indicators and finally hour of the day indicators so that we better capture the intricacies

# Add Weekend Indicator
merged_df['Weekend'] = merged_df['Timestamp'].dt.dayofweek >= 5

# Add Seasonality Indicator
def get_season(month):
    if month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Fall'
    else:
        return 'Winter'
merged_df['Season'] = merged_df['Timestamp'].dt.month.apply(get_season)
merged_df = pd.get_dummies(merged_df, columns=['Season'], drop_first=False)

# Add Hour Indicator
merged_df['Hour'] = merged_df['Timestamp'].dt.hour
merged_df = pd.get_dummies(merged_df, columns=['Hour'], prefix='Hour', drop_first=False)

In [ ]:
for column in merged_df.columns :
    print (column)

In [12]:
# Saving the generated dataset for daily charging data 

file_path = "charging_per_day.csv"
merged_df.to_csv(file_path, index=False) 